# NCBI Genomes - Oracle Open Data


The NCBI Genomes data repository contains sequence data for all single organism genome assemblies contained in [NCBI's Assembly resource](www.ncbi.nlm.nih.gov/assembly/). Please note that as of August 2022, Oracle Open Data does not include data files for refseq, so those will not be accesssible through Object Storage. They can still be downloaded by FTP from the NCBI.

The data is divided into 3 main repositories: genbank, refseq, and all.

* **Genbank** - [GenBank](https://www.ncbi.nlm.nih.gov/genbank/) is the NIH genetic sequence database, a collection of all publicly available DNA sequences. It includes primary submissions of assembled genome sequence and associated annotation data. This collection includes genome sequence data for a larger number of organisms than RefSeq directory, but some assemblies are unannotated. 

* **Refseq** - [RefSeq](https://www.ncbi.nlm.nih.gov/refseq/about/)Includes assembled genome sequence and RefSeq annotation data. All prokaryotic and eukaryotic genomes in this directory have annotation. The annotation data is either collected from NCBI annotation pipeines or the GenBank submission. This collection includes fewer organisms than GenBank, because not all genome assemblies are selected for the RefSeq project. 

* **All** - The combination of both GenBank and RefSeq assemblies. 

All files are available by anonymous file transfer protocol (FTP), please see [FTP FAQ page](https://www.ncbi.nlm.nih.gov/genome/doc/ftpfaq/). For example, to download the feature table file for the bacteria *Bacillus thuringiensis*, use `wget` followed by the ftp link as follows: `wget ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/Bacillus_thuringiensis/all_assembly_versions/GCA_000008505.1_ASM850v1_feature_table.txt.gz` in the terminal.


After division of the files into the 3 main repositories (all, genbank refseq), the files are further organized under a series of different sub-directories. Subdirectories for genbank include:

    a. archaea
    b. bacteria
    c. fungi
    d. invertebrate
    e. metagenomes
    f. other -  this directory includes synthetic genomes
    g. plant
    h. protozoa
    i. vertebrate_mammalian
    j. vertebrate_other
    k. viral
    
 
The sub-directories for refseq are as follows:

    a. archaea
    b. bacteria
    c. fungi
    d. invertebrate
    e. plant
    f. protozoa
    g. vertebrate_mammalian
    h. vertebrate_other 
    i. viral
    j. mitochondrion 
    k. plasmid     
    l. plastid 
    
Data are further organized within each of the above directories using the species binomial name. For example, *E. coli* files under genbank would have the file directory 'ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/Escherichia_coli/' , while files for humans under refseq would have the file directory 'ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/vertebrate_mammalian/Homo_sapiens/' .



## Getting all Files for An Organism 

Because there are millions of files in this repository, parsing through it manually to find files related to a specific organism would be difficult. The first function we will use will parse through the file names and identify those related to a specific organism. For this, we will be using the Oracle Command-Line-Interface (CLI). Please note that using the CLI requires that you have an oci config file set up and configured. You will also need the repository of choice (genbank, refseq, or all), sub-directory (from the lettered lists above), and species name (e.g. *Bacillus thuringiensis*), which you should input separated by an underspace (e.g. "Bacillus_thuringiensis").

In [3]:
#import oci module
import oci

# Initialize service client with default config file
config = oci.config.from_file()
object_storage_client = oci.object_storage.ObjectStorageClient(config)

#creating an empty list to store the files
list_files = []

# function to list files, takes name of repo, subdir, and species 
def list_files_for_organism(repository: str, subdir:str, species:str ) :
    myprefix = repository + "/" + subdir + "/" + species + "/" 
    
    # print prefix so user can check if input is correct
    print(myprefix)
    
    # using cli list_objects function
    list_objects_response = object_storage_client.list_objects(
        namespace_name="idcxvbiyd8fn",  # namespace for ncbi_genomes
        bucket_name="ncbi_genomes",  # bucket name
        fields = "name",
        prefix = myprefix # user user-input prefix to find species of interest
    )
    
    list_files = []
    
    # Get the data from response object, insert file names into list
    for obj in list_objects_response.data.objects:
        list_files.append(obj.name)
        
    #remore redundant file prefix
    length = len(myprefix)
    for count, name in enumerate(list_files):
        list_files[count] = name[length:]

    print('\n')
    print("The following is a list of all available files for: " + myprefix)
    print('\n')
    print(*list_files, sep = '\n')
    print('\n')
        
    

Now that the function to list files is written, let's test for a couple common bacterial strains first:

In [4]:
# Running a test for a couple strans of bacteria
list_files_for_organism("genbank", "bacteria", "Escherichia_coli")
list_files_for_organism("genbank", "bacteria", "Listeria_monocytogenes")

genbank/bacteria/Escherichia_coli/


The following is a list of all available files for: genbank/bacteria/Escherichia_coli/


README.txt
all_assembly_versions/README.txt
annotation_hashes.txt
assembly_summary.txt
assembly_summary_historical.txt
latest_assembly_versions/README.txt
reference/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_assembly_report.txt
reference/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_assembly_stats.txt
reference/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_cds_from_genomic.fna.gz
reference/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_feature_count.txt.gz
reference/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_feature_table.txt.gz
reference/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_genomic.fna.gz
reference/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_genomic.gbff.gz
reference/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_genomic.gff.gz
reference/GCA_000005845.2_ASM584v2/GCA_000005845.2_ASM584v2_genomic.gtf.gz
reference/GCA

The first few files in each list are informative summaries or descriptions of the files.The next hierarchy provides access to all assemblies for the species, latest assemblies, and selected reference or representative assemblies for the species 
(if any). Within these groupings, sequence and annotation (and other) data is provided per assembly in a series of directories that are named using the rule: `[Assembly accession.version]_[assembly name]`

For example, the directory hierarchy for the GenBank Bacillus thuringiensis strain 97-27 genome, which has the assembly accession `GCA_000008505.1` and default assembly name `ASM850v1` looks like this: `/genomes/genbank/bacteria/Bacillus_thuringiensis/all_assembly_versions/GCA_000008505.1_ASM850v1`

The directory hierarchy for the RefSeq annotated human reference genome which has the assembly accession `GCF_000001405.39` and assembly name GRCh38.p13 looks like this:
`/genomes/refseq/vertebrate_mammalian/Homo_sapiens/all_assembly_versions/GCF_000001405.39_GRCh38.p13`

Species that have been annotated by the NCBI Eukaryotic Genome Annotation Pipeline will also have a directory named `annotation_releases`. Genome assemblies of interest within `annotation_releases` can be identified using the [NCBI Assembly resource](www.ncbi.nlm.nih.gov/assembly), or by using the assembly summary report files that are provided for both all genbank and all refseq assemblies:
- ftp://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_genbank.txt
or ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/assembly_summary_genbank.txt

- ftp://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_refseq.txt
or ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/assembly_summary_refseq.txt



As mentioned earlier, any of these files can be downloaded using FTP. The syntax would be `wget 'ftp://ftp.ncbi.nlm.nih.gov/genomes` + the prefix you used to search + the file of interest. For example, use `wget 'ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/Listeria_monocytogenes/all_assembly_versions/README.txt` to get the README file.


In [11]:
!wget 'ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/Listeria_monocytogenes/all_assembly_versions/README.txt'

--2022-08-18 22:25:47--  ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/Listeria_monocytogenes/all_assembly_versions/README.txt
           => ‘README.txt’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.12, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/genbank/bacteria/Listeria_monocytogenes/all_assembly_versions ... done.
==> SIZE README.txt ... 276
==> PASV ... done.    ==> RETR README.txt ... done.
Length: 276 (unauthoritative)

100%[======================================>] 276         --.-K/s   in 0.04s   

2022-08-18 22:25:47 (7.08 KB/s) - ‘README.txt’ saved [276]

